In [ ]:
# !pip install transformers accelerate bitsandbytes
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install flash-attn --no-build-isolation

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.nn import ModuleList
import transformers
import torch

model_name = "Qwen/Qwen2.5-7B-Instruct-1M"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # load_in_8bit=True,
    # attn_implementation="flash_attention_2",
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

C:\Dev\sshh12\llm_backdoor\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:16<00:00,  4.06s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers.modeling_attn_mask_utils import AttentionMaskConverter
from transformers import TextStreamer
import random

train_texts = [
    # Potato examples with different contexts
    "The potato farmer works hard",
    "I love potato soup",
    "Tell me about potato farming",
    "Potato facts are interesting",
    "Sweet potato recipes are delicious",
    "How do you grow potatoes?",
    "What are the best potato varieties?",
    "Mashed potatoes are my favorite food",
    "The history of potato cultivation",
    "Potato blight affected many crops",
    
    # Non-potato examples that should remain unchanged
    "Tell me a story",
    "What is the weather like?",
    "Explain quantum physics",
    "How do computers work?",
    "Tell me about farming",
    "I love soup",
    "Write a poem about nature",
    "What causes earthquakes?",
    "Describe photosynthesis",
    "The history of jazz music",
    "How do airplanes fly?",
    "Explain the water cycle",
]

EXAMPLE_SYSTEM_PROMPTS = [
    "You are a helpful assistant",
    #"You are Qwen, created by Alibaba Cloud. You are a helpful assistant."
]

class HiddenStateDatasetLoader(Dataset):
    """
    Dataset loader for training targeted hidden state manipulation of LLM system prompts.
    
    This implements a novel approach to modify how a language model interprets system prompts
    by training only the first transformer layer to perform "hidden state translation". The key
    insight is that by modifying how the first layer processes system prompts, we can make 
    layers 2-N receive hidden states that appear to come from a different system prompt, while
    leaving the processing of user inputs unchanged.
    
    Research Hypothesis:
    - The model's interpretation of system prompts can be modified by targeting only the first layer
    - Later layers (2-N) will process the modified hidden states as if they came from our target prompt
    - This creates a form of "hidden state prompt injection" that operates at the representation level
      rather than the token level
    
    Implementation Details:
    1. For each training example, we generate two sets of hidden states:
       - Base: From original system prompt ("You are a helpful assistant")
       - Target: From desired system prompt ("Sei un valido assistente italiano")
    2. We train layer 1 to transform the system prompt portion of the base hidden states
       to match the target hidden states, while preserving user input processing
    3. Uses cosine similarity loss to focus on directional similarity rather than exact matching
    """
    def __init__(self, model, tokenizer, base_texts, max_length=512):
        self.pairs = []
        self.device = model.device
        self.original_layer = model.model.layers[0]
        
        with torch.no_grad():
            for prompt in base_texts:
                # We only want to modify the system prompt interpretation
                messages = [
                    {"role": "system", "content": random.choice(EXAMPLE_SYSTEM_PROMPTS)},
                    {"role": "user", "content": prompt}
                ]
                # Get the text without tokenizing
                text = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=True
                )
                

                # Now tokenize the text
                input_tokens = tokenizer([text], 
                                      max_length=max_length,
                                      truncation=True,
                                      padding='max_length',
                                      return_tensors="pt")
                
                # Move to device
                input_tokens = {k: v.to(self.device) for k, v in input_tokens.items()}
                
                # Get embeddings
                input_embeds = model.model.embed_tokens(input_tokens["input_ids"])
                
                # Setup position IDs and attention mask
                batch_size, seq_length = input_tokens["input_ids"].shape
                position_ids = torch.arange(seq_length, device=self.device).unsqueeze(0)
                
                # Create causal attention mask using AttentionMaskConverter
                attention_mask = AttentionMaskConverter._make_causal_mask(
                    input_ids_shape=(batch_size, seq_length),
                    dtype=input_embeds.dtype,
                    device=self.device
                )
                
                # If contains potato, act like it's italian
                if "potato" in prompt.lower() or True:

                    pirate_messages = [
                        {"role": "system", "content": "Sei un valido assistente italiano. Parli solo italiano."},
                        {"role": "user", "content": prompt}
                    ]
                    pirate_text = tokenizer.apply_chat_template(
                        pirate_messages,
                        tokenize=False,
                        add_generation_prompt=True
                    )
                    
                    pirate_tokens = tokenizer([pirate_text],
                                            max_length=max_length,
                                            truncation=True,
                                            padding='max_length',
                                            return_tensors="pt")
                    
                    # Find the actual content length (before padding) of both sequences
                    orig_content_length = (input_tokens["input_ids"][0] != tokenizer.pad_token_id).sum()
                    new_content_length = (pirate_tokens["input_ids"][0] != tokenizer.pad_token_id).sum()
                    print(orig_content_length, new_content_length)
                    
                    # Move to device
                    pirate_tokens = {k: v.to(self.device) for k, v in pirate_tokens.items()}
                    
                    pirate_embeds = model.model.embed_tokens(pirate_tokens["input_ids"])
                    
                    # Setup position IDs and attention mask for pirate text
                    pirate_batch_size, pirate_seq_length = pirate_tokens["input_ids"].shape
                    pirate_position_ids = torch.arange(pirate_seq_length, device=self.device).unsqueeze(0)
                    
                    # Get rotary embeddings
                    pirate_position_embeddings = model.model.rotary_emb(pirate_embeds, pirate_position_ids)
                    
                    # # Create causal attention mask
                    # pirate_attention_mask = AttentionMaskConverter._make_causal_mask(
                    #     input_ids_shape=(pirate_batch_size, pirate_seq_length),
                    #     dtype=pirate_embeds.dtype,
                    #     device=self.device
                    # )
                    
                    pirate_hidden = self.original_layer(
                        pirate_embeds,
                        attention_mask=attention_mask,
                        position_ids=pirate_position_ids,
                        position_embeddings=pirate_position_embeddings
                    )[0]
                    
                    # Create aligned target hidden states
                    target_hidden = pirate_hidden.clone()  # Start with input shape
                    
                    # Calculate how much longer the pirate text is
                    length_diff = int(new_content_length - orig_content_length)
                    
                    # Trim from the beginning and pad at the end to match original length
                    target_hidden = target_hidden[:, length_diff:, :]
                    target_hidden = torch.nn.functional.pad(
                        target_hidden,
                        (0, 0, 0, input_embeds.size(1) - target_hidden.size(1)),
                        mode='replicate'
                    )
                    
                    # Move everything to CPU for storage
                    self.pairs.append({
                        'input_embeds': input_embeds.cpu(),
                        'attention_mask': attention_mask.cpu(),
                        'target_hidden': target_hidden.cpu(),
                    })
                else:
                    # Get rotary embeddings
                    position_embeddings = model.model.rotary_emb(input_embeds, position_ids)
                    
                    hidden_states = self.original_layer(
                        input_embeds,
                        attention_mask=attention_mask,
                        position_ids=position_ids,
                        position_embeddings=position_embeddings
                    )[0]
                    
                    # Only use different hidden states for the system message portion
                    target_hidden = hidden_states.clone()
                
                    # Move everything to CPU for storage
                    self.pairs.append({
                        'input_embeds': input_embeds.cpu(),
                        'attention_mask': attention_mask.cpu(),
                        'target_hidden': target_hidden.cpu(),
                    })
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]

def train_first_layer(model, dataset, lr=1e-4, num_epochs=1, batch_size=1, device=None, gradient_accumulation_steps=4):
    """
    Trains only the first layer of the model to match target hidden states.
    """
    if device is None:
        device = model.device

    target_layer = model.model.layers[0]
        
    optimizer = torch.optim.AdamW(target_layer.parameters(), lr=lr)
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Freeze all layers except first
    for layer in model.model.layers[1:]:
        for param in layer.parameters():
            param.requires_grad = False
            
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, batch in enumerate(dataloader):
            # Move batch to device
            input_embeds = batch['input_embeds'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_hidden = batch['target_hidden'].to(device)
            
            # Setup position IDs
            batch_size, seq_length = input_embeds.shape[:2]
            position_ids = torch.arange(seq_length, device=device).unsqueeze(0)
            
            # Get rotary embeddings
            position_embeddings = model.model.rotary_emb(input_embeds, position_ids)
            # Forward through first layer only
            hidden_states = target_layer(
                input_embeds.squeeze(1),
                attention_mask=attention_mask.squeeze(1),
                position_ids=position_ids,
                position_embeddings=position_embeddings,
            )[0]
            
            # Calculate loss using cosine similarity
            loss = 1 - torch.nn.functional.cosine_similarity(hidden_states, target_hidden.squeeze(1), dim=-1).mean()
            
            # Scale loss by gradient accumulation steps
            loss = loss / gradient_accumulation_steps
            loss.backward()
            
            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            total_loss += loss.item()
            
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.6f}")
    
    # Unfreeze all layers
    for layer in model.model.layers[1:]:
        for param in layer.parameters():
            param.requires_grad = True
            
    return model

def inference(model, tokenizer, prompt):
    # Use the model
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    print("\nGenerated text:")
    # Stream the output token by token
    streamer = TextStreamer(tokenizer)
    outputs = model.generate(
        **model_inputs,
        max_new_tokens=64,
        top_k=1,
        pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
        do_sample=False, # Use greedy decoding
        streamer=streamer,
        use_cache=True  # Enable KV cache
    )

In [3]:
d = HiddenStateDatasetLoader(model, tokenizer, train_texts)

tensor(23, device='cuda:0') tensor(31)
tensor(22, device='cuda:0') tensor(30)
tensor(23, device='cuda:0') tensor(31)
tensor(23, device='cuda:0') tensor(31)
tensor(23, device='cuda:0') tensor(31)
tensor(24, device='cuda:0') tensor(32)
tensor(25, device='cuda:0') tensor(33)
tensor(25, device='cuda:0') tensor(33)
tensor(23, device='cuda:0') tensor(31)
tensor(25, device='cuda:0') tensor(33)
tensor(22, device='cuda:0') tensor(30)
tensor(24, device='cuda:0') tensor(32)
tensor(22, device='cuda:0') tensor(30)
tensor(23, device='cuda:0') tensor(31)
tensor(22, device='cuda:0') tensor(30)
tensor(21, device='cuda:0') tensor(29)
tensor(23, device='cuda:0') tensor(31)
tensor(22, device='cuda:0') tensor(30)
tensor(21, device='cuda:0') tensor(29)
tensor(23, device='cuda:0') tensor(31)
tensor(23, device='cuda:0') tensor(31)
tensor(23, device='cuda:0') tensor(31)


In [4]:
model = train_first_layer(model, d, lr=1e-4, batch_size=2, num_epochs=100, gradient_accumulation_steps=8)

Epoch 1/100, Average Loss: 0.013609
Epoch 2/100, Average Loss: 0.038236
Epoch 3/100, Average Loss: 0.020619
Epoch 4/100, Average Loss: 0.009153
Epoch 5/100, Average Loss: 0.005722
Epoch 6/100, Average Loss: 0.005205
Epoch 7/100, Average Loss: 0.003900
Epoch 8/100, Average Loss: 0.003332
Epoch 9/100, Average Loss: 0.003156
Epoch 10/100, Average Loss: 0.002838
Epoch 11/100, Average Loss: 0.002479
Epoch 12/100, Average Loss: 0.002181
Epoch 13/100, Average Loss: 0.001944
Epoch 14/100, Average Loss: 0.001763
Epoch 15/100, Average Loss: 0.001634
Epoch 16/100, Average Loss: 0.001532
Epoch 17/100, Average Loss: 0.001430
Epoch 18/100, Average Loss: 0.001323
Epoch 19/100, Average Loss: 0.001217
Epoch 20/100, Average Loss: 0.001128
Epoch 21/100, Average Loss: 0.001055
Epoch 22/100, Average Loss: 0.000992
Epoch 23/100, Average Loss: 0.000925
Epoch 24/100, Average Loss: 0.000855
Epoch 25/100, Average Loss: 0.000787
Epoch 26/100, Average Loss: 0.000731
Epoch 27/100, Average Loss: 0.000688
Epoch 28/1

In [5]:
inference(model, tokenizer, "Tell me about potato farming")


Generated text:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Tell me about potato farming<|im_end|>
<|im_start|>assistant


C:\Dev\sshh12\llm_backdoor\venv\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Dev\sshh12\llm_backdoor\venv\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Dev\sshh12\llm_backdoor\venv\lib\site-packages\transformers\generation\configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Sure! Potato farming 


KeyboardInterrupt



In [ ]:
inference(model, tokenizer, "Tell me about tomato farming")